In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns


def load_file(filepath):
    dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values


# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = np.dstack(loaded)
    return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    #filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + '../../dataset/UCI_HAR/UCI HAR Dataset/')
    # load all test
    testX, testy = load_dataset_group('test', prefix + '../../dataset/UCI_HAR/UCI HAR Dataset/')
    
    #zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    #one hot encode y
    trainy_one_hot = to_categorical(trainy)
    testy_one_hot = to_categorical(testy)
    print(trainX.shape, trainy.shape, trainy_one_hot.shape, testX.shape, testy.shape, testy_one_hot.shape)
    return trainX, trainy, trainy_one_hot, testX, testy, testy_one_hot

In [2]:
trainX, trainy, trainy_one_hot, testX, testy, testy_one_hot = load_dataset()

(7352, 128, 6) (7352, 1) (7352, 6) (2947, 128, 6) (2947, 1) (2947, 6)


In [3]:
print("X train shape: ", trainX.shape)
print("Y train shape: ", trainy.shape)
print("Y train One hot shape: ", trainy_one_hot.shape)
print("X test shape: ", testX.shape)
print("Y test shape: ", testy.shape)
print("Y test One hot shape: ", testy_one_hot.shape)

X train shape:  (7352, 128, 6)
Y train shape:  (7352, 1)
Y train One hot shape:  (7352, 6)
X test shape:  (2947, 128, 6)
Y test shape:  (2947, 1)
Y test One hot shape:  (2947, 6)


In [4]:
print("0: Walking:" ,np.where(trainy == 0)[0].size)
print("1: WU:" ,np.where(trainy == 1)[0].size)
print("2: WD:" ,np.where(trainy == 2)[0].size)
print("3: Sitting:" ,np.where(trainy == 3)[0].size)
print("4: Standing:" ,np.where(trainy == 4)[0].size)
print("5: Laying:" ,np.where(trainy == 5)[0].size)

0: Walking: 1226
1: WU: 1073
2: WD: 986
3: Sitting: 1286
4: Standing: 1374
5: Laying: 1407


In [5]:
unique, counts = np.unique(trainy, return_counts=True)
print ("Train data label statistics::")
print (np.asarray((unique, counts)).T)  

unique, counts = np.unique(testy, return_counts=True)
print ("Test data label statistics::")
print (np.asarray((unique, counts)).T)   

Train data label statistics::
[[   0 1226]
 [   1 1073]
 [   2  986]
 [   3 1286]
 [   4 1374]
 [   5 1407]]
Test data label statistics::
[[  0 496]
 [  1 471]
 [  2 420]
 [  3 491]
 [  4 532]
 [  5 537]]


In [6]:
X_train_all = trainX   # at this stage, the data includes both dynamic and static HAR data
y_train_all = trainy

X_test_all = testX
y_test_all = testy

In [7]:
X_train_all = (X_train_all - X_train_all.min()) / (X_train_all.max() - X_train_all.min())
X_test_all = (X_test_all - X_test_all.min()) / (X_test_all.max() - X_test_all.min())

X_train_all = X_train_all.astype('float32')
X_test_all = X_test_all.astype('float32')

In [8]:
import random

static_2 = np.where(trainy  == 0)[0]
static_3 = np.where(trainy == 1)[0]
static_4 = np.where(trainy  == 2)[0]

dynamic_3 = np.where(trainy == 3)[0]
dynamic_4 = np.where(trainy == 4)[0]
dynamic_5 = np.where(trainy == 5)[0]

static = np.concatenate([static_2, static_3, static_4])
static_list = static.tolist()

dynamic = np.concatenate([dynamic_3, dynamic_4, dynamic_5])
dynamic_list = dynamic.tolist()

# Shuffle dynamic data index
r = random.random()
random.shuffle(static_list, lambda: r)
random.shuffle(dynamic_list, lambda: r)

static = np.array(static_list)
dynamic = np.array(dynamic_list)

trainX_static = X_train_all[static]
trainy_static = y_train_all[static]

trainX_dynamic = X_train_all[dynamic]
trainy_dynamic = y_train_all[dynamic]

trainX_combined = X_train_all
trainy_combined = y_train_all

/var/folders/hf/lrx32m9n0n7f1twmsv3x62gw0000gn/T/ipykernel_67257/1208802571.py:19: DeprecationWarning: The *random* parameter to shuffle() has been deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.shuffle(static_list, lambda: r)
/var/folders/hf/lrx32m9n0n7f1twmsv3x62gw0000gn/T/ipykernel_67257/1208802571.py:20: DeprecationWarning: The *random* parameter to shuffle() has been deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.shuffle(dynamic_list, lambda: r)


In [9]:
static_2 = np.where(testy == 0)[0]
static_3 = np.where(testy == 1)[0]
static_4 = np.where(testy == 2)[0]
static = np.concatenate([static_2, static_3, static_4])
static_list = static.tolist()

dynamic_3 = np.where(testy == 3)[0]
dynamic_4 = np.where(testy == 4)[0]
dynamic_5 = np.where(testy == 5)[0]
dynamic = np.concatenate([dynamic_3, dynamic_4, dynamic_5])
dynamic_list = dynamic.tolist()

r = random.random()
random.shuffle(static_list, lambda: r)
random.shuffle(dynamic_list, lambda: r)

static = np.array(static_list)
dynamic = np.array(dynamic_list)

testX_static = X_test_all[static]
testy_static = y_test_all[static]

testX_dynamic = X_test_all[dynamic]
testy_dynamic = y_test_all[dynamic]

testX_combined = X_test_all
testy_combined = y_test_all

/var/folders/hf/lrx32m9n0n7f1twmsv3x62gw0000gn/T/ipykernel_67257/3041243064.py:14: DeprecationWarning: The *random* parameter to shuffle() has been deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.shuffle(static_list, lambda: r)
/var/folders/hf/lrx32m9n0n7f1twmsv3x62gw0000gn/T/ipykernel_67257/3041243064.py:15: DeprecationWarning: The *random* parameter to shuffle() has been deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.shuffle(dynamic_list, lambda: r)


In [10]:
print("Train 0_s: Walking:" ,np.where(testy_static == 0)[0].size)
print("Train1_s: WU:" ,np.where(testy_static == 1)[0].size)
print("Train2_s: WD:" ,np.where(testy_static == 2)[0].size)
print("Train3_s: Sitting:" ,np.where(testy_static == 3)[0].size)
print("Train4_s: Standing:" ,np.where(testy_static == 4)[0].size)
print("Train5_s: Laying:" ,np.where(testy_static == 5)[0].size) 

print("Test 0_s: Walking:" ,np.where(trainy_static == 0)[0].size)
print("Test 1_s: WU:" ,np.where(trainy_static == 1)[0].size)
print("Test 2_s: WD:" ,np.where(trainy_static == 2)[0].size)
print("Test 3_s: Sitting:" ,np.where(trainy_static == 3)[0].size)
print("Test 4_s: Standing:" ,np.where(trainy_static == 4)[0].size)
print("Test 5_s: Laying:" ,np.where(trainy_static == 5)[0].size) 

Train 0_s: Walking: 496
Train1_s: WU: 471
Train2_s: WD: 420
Train3_s: Sitting: 0
Train4_s: Standing: 0
Train5_s: Laying: 0
Test 0_s: Walking: 1226
Test 1_s: WU: 1073
Test 2_s: WD: 986
Test 3_s: Sitting: 0
Test 4_s: Standing: 0
Test 5_s: Laying: 0


In [11]:
print("Train 0_d: Walking:" ,np.where(testy_dynamic == 0)[0].size)
print("Train1_d: WU:" ,np.where(testy_dynamic == 1)[0].size)
print("Train2_d: WD:" ,np.where(testy_dynamic == 2)[0].size)
print("Train3_d: Sitting:" ,np.where(testy_dynamic == 3)[0].size)
print("Train4_d: Standing:" ,np.where(testy_dynamic == 4)[0].size)
print("Train5_d: Laying:" ,np.where(testy_dynamic == 5)[0].size) 

print("Test 0_d: Walking:" ,np.where(trainy_dynamic == 0)[0].size)
print("Test 1_d: WU:" ,np.where(trainy_dynamic == 1)[0].size)
print("Test 2_d: WD:" ,np.where(trainy_dynamic == 2)[0].size)
print("Test 3_d: Sitting:" ,np.where(trainy_dynamic == 3)[0].size)
print("Test 4_d: Standing:" ,np.where(trainy_dynamic == 4)[0].size)
print("Test 5_d: Laying:" ,np.where(trainy_dynamic == 5)[0].size) 

Train 0_d: Walking: 0
Train1_d: WU: 0
Train2_d: WD: 0
Train3_d: Sitting: 491
Train4_d: Standing: 532
Train5_d: Laying: 537
Test 0_d: Walking: 0
Test 1_d: WU: 0
Test 2_d: WD: 0
Test 3_d: Sitting: 1286
Test 4_d: Standing: 1374
Test 5_d: Laying: 1407


In [12]:
trainy_static_one_hot = to_categorical(trainy_static)
testy_static_one_hot = to_categorical(testy_static)

trainy_dynamic_one_hot = to_categorical(trainy_dynamic)
testy_dynamic_one_hot = to_categorical(testy_dynamic)

trainy_combined_one_hot = to_categorical(trainy)
testy_combined_one_hot = to_categorical(testy)

In [13]:
print("X train shape: ", trainX_static.shape)
print("Y train shape: ", trainy_static.shape)
print("Y train One hot shape: ", trainy_static_one_hot.shape)
print("X test shape: ", testX_static.shape)
print("Y test shape: ", testy_static.shape)
print("Y test One hot shape: ", testy_static_one_hot.shape)

X train shape:  (3285, 128, 6)
Y train shape:  (3285, 1)
Y train One hot shape:  (3285, 3)
X test shape:  (1387, 128, 6)
Y test shape:  (1387, 1)
Y test One hot shape:  (1387, 3)


In [14]:
print("X train shape: ", trainX_dynamic.shape)
print("Y train shape: ", trainy_dynamic.shape)
print("Y train One hot shape: ", trainy_dynamic_one_hot.shape)
print("X test shape: ", testX_dynamic.shape)
print("Y test shape: ", testy_dynamic.shape)
print("Y test One hot shape: ", testy_dynamic_one_hot.shape)

X train shape:  (4067, 128, 6)
Y train shape:  (4067, 1)
Y train One hot shape:  (4067, 6)
X test shape:  (1560, 128, 6)
Y test shape:  (1560, 1)
Y test One hot shape:  (1560, 6)


In [15]:
print("X train shape: ", trainX_combined.shape)
print("Y train shape: ", trainy_combined.shape)
print("Y train One hot shape: ", trainy_combined_one_hot.shape)
print("X test shape: ", testX_combined.shape)
print("Y test shape: ", testy_combined.shape)
print("Y test One hot shape: ", testy_combined_one_hot.shape)

X train shape:  (7352, 128, 6)
Y train shape:  (7352, 1)
Y train One hot shape:  (7352, 6)
X test shape:  (2947, 128, 6)
Y test shape:  (2947, 1)
Y test One hot shape:  (2947, 6)


In [16]:
unique, counts = np.unique(trainy_static, return_counts=True)
print ("Train data label statistics::")
print (np.asarray((unique, counts)).T)  

unique, counts = np.unique(testy_static, return_counts=True)
print ("Test data label statistics::")
print (np.asarray((unique, counts)).T)  

Train data label statistics::
[[   0 1226]
 [   1 1073]
 [   2  986]]
Test data label statistics::
[[  0 496]
 [  1 471]
 [  2 420]]


In [17]:
unique, counts = np.unique(trainy_dynamic, return_counts=True)
print ("Train data label statistics::")
print (np.asarray((unique, counts)).T)  

unique, counts = np.unique(testy_dynamic, return_counts=True)
print ("Test data label statistics::")
print (np.asarray((unique, counts)).T)  

Train data label statistics::
[[   3 1286]
 [   4 1374]
 [   5 1407]]
Test data label statistics::
[[  3 491]
 [  4 532]
 [  5 537]]


In [18]:
unique, counts = np.unique(trainy_combined, return_counts=True)
print ("Train data label statistics::")
print (np.asarray((unique, counts)).T)  

unique, counts = np.unique(testy_combined, return_counts=True)
print ("Test data label statistics::")
print (np.asarray((unique, counts)).T)  

Train data label statistics::
[[   0 1226]
 [   1 1073]
 [   2  986]
 [   3 1286]
 [   4 1374]
 [   5 1407]]
Test data label statistics::
[[  0 496]
 [  1 471]
 [  2 420]
 [  3 491]
 [  4 532]
 [  5 537]]


In [19]:
from sklearn.model_selection import train_test_split

X_train_static,X_val_static,y_train_static_one_hot,y_val_static_one_hot,y_train_static,y_val_static=train_test_split(trainX_static, trainy_static_one_hot, trainy_static,test_size=0.2,random_state=100)

In [20]:
from sklearn.model_selection import train_test_split

X_train_dynamic,X_val_dynamic,y_train_dynamic_one_hot,y_val_dynamic_one_hot,y_train_dynamic,y_val_dynamic=train_test_split(trainX_dynamic, trainy_dynamic_one_hot, trainy_dynamic,test_size=0.2,random_state=100)

In [21]:
from sklearn.model_selection import train_test_split

X_train_combined,X_val_combined,y_train_combined_one_hot,y_val_combined_one_hot,y_train_combined,y_val_combined=train_test_split(trainX_combined, trainy_combined_one_hot, trainy_combined,test_size=0.2,random_state=100)

In [22]:
X_train_combined.shape, y_train_combined.shape, testX_combined.shape, testy_combined.shape

((5881, 128, 6), (5881, 1), (2947, 128, 6), (2947, 1))

In [23]:
n_timesteps_dynamic, n_features_dynamic, n_outputs_dynamic = X_train_dynamic.shape[1], X_train_dynamic.shape[2], testy_dynamic_one_hot.shape[1]


In [24]:
n_timesteps_static, n_features_static, n_outputs_static = X_train_static.shape[1], X_train_static.shape[2], testy_static_one_hot.shape[1]

In [25]:
n_timesteps_combined, n_features_combined, n_outputs_combined = X_train_combined.shape[1], X_train_combined.shape[2], testy_combined_one_hot.shape[1]


In [26]:
n_timesteps_static, n_features_static, n_outputs_static, n_timesteps_dynamic, n_features_dynamic, n_outputs_dynamic, n_timesteps_combined, n_features_combined, n_features_combined

(128, 6, 3, 128, 6, 6, 128, 6, 6)

In [43]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, InputLayer, Dropout, Flatten, BatchNormalization, Conv1D, GlobalAveragePooling1D
from tensorflow.keras.layers import concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD
from keras.utils import to_categorical

# Model

In [28]:
learning_rate = 0.0001
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum,  nesterov=False)

/Users/kimtaeyoon/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [29]:
def L1C1(n_timesteps, n_features, n_outputs):
    model = Sequential([
        LSTM(64, input_shape=(n_timesteps, n_features), return_sequences=True, activation='relu'),
        Dropout(0.3),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        GlobalAveragePooling1D(),
        BatchNormalization(epsilon=1e-06),
        Dense(n_outputs, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [30]:
def L1C2(n_timesteps, n_features, n_outputs):
    model = Sequential([
        LSTM(64, input_shape=(n_timesteps, n_features), return_sequences=True, activation='relu'),
        Dropout(0.3),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(5),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        GlobalAveragePooling1D(),
        BatchNormalization(epsilon=1e-06),
        Dense(n_outputs, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [31]:
def L2C1(n_timesteps, n_features, n_outputs):
    model = Sequential([
        LSTM(64, input_shape=(n_timesteps, n_features), return_sequences=True, activation='relu'),
        Dropout(0.3),
        LSTM(64, return_sequences=True, activation='relu'),
        Dropout(0.3),
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)),
        GlobalAveragePooling1D(),
        BatchNormalization(epsilon=1e-06),
        Dense(n_outputs, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [32]:
def L2C2(n_timesteps, n_features, n_outputs):
    model = Sequential([
        LSTM(64, input_shape=(n_timesteps, n_features), return_sequences=True, activation='relu'),
        Dropout(0.3),
        LSTM(64, activation='relu', return_sequences=True),
        Dropout(0.3),
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)),
        MaxPooling1D(5),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        GlobalAveragePooling1D(),
        BatchNormalization(epsilon=1e-06),
        Dense(n_outputs, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [33]:
import os
MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

# Static

In [34]:
def make_static_history(model, filename):
    model_path = MODEL_SAVE_FOLDER_PATH + f'{filename}_static' + '-{epoch:02d}-{val_loss:.4f}.hdf5'
    cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
    return model.fit(x=X_train_static, y=y_train_static_one_hot, epochs=50, batch_size=32, validation_data=(X_val_static, y_val_static_one_hot), callbacks=[cb_checkpoint])

In [35]:
X_train_static.shape, y_train_static_one_hot.shape, X_val_static.shape, y_val_static_one_hot.shape

((2628, 128, 6), (2628, 3), (657, 128, 6), (657, 3))

In [36]:
# fit network
L1C1_model_static = L1C1(n_timesteps_static, n_features_static, n_outputs_static)
L1C2_model_static = L1C2(n_timesteps_static, n_features_static, n_outputs_static)
L2C1_model_static = L2C1(n_timesteps_static, n_features_static, n_outputs_static)
L2C2_model_static = L2C2(n_timesteps_static, n_features_static, n_outputs_static)

L1C1_history_static = make_static_history(L1C1_model_static, "L1C1")
L1C2_history_static = make_static_history(L1C2_model_static, "L1C2")
L2C1_history_static = make_static_history(L2C1_model_static, "L2C1")
L2C2_history_static = make_static_history(L2C2_model_static, "L2C2")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128, 64)           18176     
                                                                 
 dropout (Dropout)           (None, 128, 64)           0         
                                                                 
 conv1d (Conv1D)             (None, 126, 64)           12352     
                                                                 
 global_average_pooling1d (G  (None, 64)               0         
 lobalAveragePooling1D)                                          
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 3)                 1

# Dynamic

In [44]:
def make_dynamic_history(model, filename):
    model_path = MODEL_SAVE_FOLDER_PATH + f'{filename}_dynamic' + '-{epoch:02d}-{val_loss:.4f}.hdf5'
    cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
    cb_early_stopping = EarlyStopping(monitor='val_loss', patience=20)
    return model.fit(x=X_train_dynamic, y=y_train_dynamic_one_hot, epochs=50, batch_size=32, validation_data=(X_val_dynamic, y_val_dynamic_one_hot), callbacks=[cb_checkpoint, cb_early_stopping])

In [45]:
# fit network
L1C1_model_dynamic = L1C1(n_timesteps_dynamic, n_features_dynamic, n_outputs_dynamic)
L1C2_model_dynamic = L1C2(n_timesteps_dynamic, n_features_dynamic, n_outputs_dynamic)
L2C1_model_dynamic = L2C1(n_timesteps_dynamic, n_features_dynamic, n_outputs_dynamic)
L2C2_model_dynamic = L2C2(n_timesteps_dynamic, n_features_dynamic, n_outputs_dynamic)

L1C1_history_dynamic = make_dynamic_history(L1C1_model_dynamic, "L1C1")
L1C2_history_dynamic = make_dynamic_history(L1C2_model_dynamic, "L1C2")
L2C1_history_dynamic = make_dynamic_history(L2C1_model_dynamic, "L2C1")
L2C2_history_dynamic = make_dynamic_history(L2C2_model_dynamic, "L2C2")

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_24 (LSTM)              (None, 128, 64)           18176     
                                                                 
 dropout_24 (Dropout)        (None, 128, 64)           0         
                                                                 
 conv1d_24 (Conv1D)          (None, 126, 64)           12352     
                                                                 
 global_average_pooling1d_16  (None, 64)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 batch_normalization_16 (Bat  (None, 64)               256       
 chNormalization)                                                
                                                                 
 dense_16 (Dense)            (None, 6)               

# Combined

In [50]:
def make_combined_history(model, filename):
    model_path = MODEL_SAVE_FOLDER_PATH + f'{filename}_combined' + '-{epoch:02d}-{val_loss:.4f}.hdf5'
    cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
    cb_early_stopping = EarlyStopping(monitor='val_loss', patience=20)
    return model.fit(x=X_train_combined, y=y_train_combined_one_hot, epochs=50, batch_size=32, validation_data=(X_val_combined, y_val_combined_one_hot), callbacks=[cb_checkpoint, cb_early_stopping])

In [51]:
# fit network
L1C1_model_combined = L1C1(n_timesteps_combined, n_features_combined, n_outputs_combined)
L1C2_model_combined = L1C2(n_timesteps_combined, n_features_combined, n_outputs_combined)
L2C1_model_combined = L2C1(n_timesteps_combined, n_features_combined, n_outputs_combined)
L2C2_model_combined = L2C2(n_timesteps_combined, n_features_combined, n_outputs_combined)

L1C1_history_combined = make_combined_history(L1C1_model_combined, "L1C1")
L1C2_history_combined = make_combined_history(L1C2_model_combined, "L1C2")
L2C1_history_combined = make_combined_history(L2C1_model_combined, "L2C1")
L2C2_history_combined = make_combined_history(L2C2_model_combined, "L2C2")

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_36 (LSTM)              (None, 128, 64)           18176     
                                                                 
 dropout_36 (Dropout)        (None, 128, 64)           0         
                                                                 
 conv1d_36 (Conv1D)          (None, 126, 64)           12352     
                                                                 
 global_average_pooling1d_24  (None, 64)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 batch_normalization_24 (Bat  (None, 64)               256       
 chNormalization)                                                
                                                                 
 dense_24 (Dense)            (None, 6)               

# Result

In [ ]:
def view_loss(model_static_history, model_dynamic_history, model_combined_history, name):
    loss_static = model_static_history.history['loss']
    loss_dynamic = model_dynamic_history.history['loss']
    loss_combined = model_combined_history.history['loss']
    epochs = range(1, len(loss_static) + 1)
    plt.plot(epochs, loss_static, 'y', label='STATIC Training loss')
    plt.plot(epochs, loss_dynamic, 'b', label='DYNAMIC Training loss')
    plt.plot(epochs, loss_combined, color='violet', label='COMBINED Training loss')
    plt.title(f'{name} Training loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
def view_val_acc(model_static_history, model_dynamic_history, model_combined_history, name):
    val_acc_static = model_static_history.history['accuracy']
    val_acc_dynamic = model_dynamic_history.history['accuracy']
    val_acc_combined = model_combined_history.history['accuracy']
    epochs = range(1, len(val_acc_static) + 1)
    plt.plot(epochs, val_acc_static, 'y', label='STATIC Training loss')
    plt.plot(epochs, val_acc_dynamic, 'b', label='DYNAMIC Training loss')
    plt.plot(epochs, val_acc_combined, color='violet', label='COMBINED Training loss')
    plt.title(f'{name} Validation Acc')
    plt.xlabel('Epochs')
    plt.ylabel('Acc')
    plt.legend()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

view_loss(L1C1_history_static, L1C1_history_dynamic, L1C1_history_combined, "L1C1")
view_loss(L1C2_history_static, L1C2_history_dynamic, L1C2_history_combined, "L1C2")
view_loss(L2C1_history_static, L2C1_history_dynamic, L2C1_history_combined, "L2C1")
view_loss(L2C2_history_static, L2C2_history_dynamic, L2C2_history_combined, "L2C2")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

view_val_acc(L1C1_history_static, L1C1_history_dynamic, L1C1_history_combined, "L1C1")
view_val_acc(L1C2_history_static, L1C2_history_dynamic, L1C2_history_combined, "L1C2")
view_val_acc(L2C1_history_static, L2C1_history_dynamic, L2C1_history_combined, "L2C1")
view_val_acc(L2C2_history_static, L2C2_history_dynamic, L2C2_history_combined, "L2C2")

In [ ]:
def heatmap_static(model):
    from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
    pred_static = model.predict(testX_static)
    pred_static = np.argmax(pred_static, axis=-1)
    cm_static=confusion_matrix(testy_static,pred_static)
    print(cm_static)
    print(accuracy_score(testy_static,pred_static))
    print(classification_report(testy_static,pred_static))
    sns.heatmap(cm_static, annot=True, fmt = '.2f')

In [ ]:
heatmap_static(L1C1_model_dynamic)

In [ ]:
def eval(model_static, model_dynamic, model_combined, x_static, y_static, x_dynamic, y_dynamic, x_combined, y_combined):
    score_static = model_static.evaluate(x_static, y_static)
    score_dynamic = model_dynamic.evaluate(x_dynamic, y_dynamic)
    score_combined = model_combined.evaluate(x_combined, y_combined)

    return score_static[1], score_dynamic[1], score_combined[1]

In [ ]:
eval(L1C1_model_static, L1C1_model_dynamic, L1C1_model_combined, testX_static, testy_static_one_hot, testX_dynamic, testy_dynamic_one_hot, testX_combined, testy_combined_one_hot)
eval(L1C2_model_static, L1C2_model_dynamic, L1C2_model_combined, testX_static, testy_static_one_hot, testX_dynamic, testy_dynamic_one_hot, testX_combined, testy_combined_one_hot)
eval(L2C1_model_static, L2C1_model_dynamic, L2C1_model_combined, testX_static, testy_static_one_hot, testX_dynamic, testy_dynamic_one_hot, testX_combined, testy_combined_one_hot)
eval(L2C2_model_static, L2C2_model_dynamic, L2C2_model_combined, testX_static, testy_static_one_hot, testX_dynamic, testy_dynamic_one_hot, testX_combined, testy_combined_one_hot)

# Accuracy
### Static + Dynamic : 0.8941
### Static : 0.9611
### Dynamic : 0.8449

In [ ]:
static_ratio = testX_static.shape[0] / testX.shape[0]
dynamic_ratio = testX_dynamic.shape[0] / testX.shape[0]

testX_static.shape[0], testX_dynamic.shape[0]

### Accuracy

In [ ]:
mod_acc = static_ratio * score_static[1] + dynamic_ratio * score_dynamic[1]
mod_acc

# Load Saved Model

In [52]:
from keras.models import load_model

In [58]:
L1C1_static = load_model('./HAR_LSTM-CNN_extended_model/L1C1_static-44-0.0163.hdf5')
L1C1_dynamic = load_model('./HAR_LSTM-CNN_extended_model/L1C1_dynamic-10-0.2147.hdf5')
L1C1_combined = load_model('./HAR_LSTM-CNN_extended_model/L1C1_combined-36-0.1303.hdf5')
L1C2_static = load_model('./HAR_LSTM-CNN_extended_model/L1C2_static-28-0.1600.hdf5')
L1C2_dynamic = load_model('./HAR_LSTM-CNN_extended_model/L1C2_dynamic-10-0.2201.hdf5')
L1C2_combined = load_model('./HAR_LSTM-CNN_extended_model/L1C2_combined-28-0.1554.hdf5')
L2C1_static = load_model('./HAR_LSTM-CNN_extended_model/L2C1_static-35-0.0124.hdf5')
L2C1_dynamic = load_model('./HAR_LSTM-CNN_extended_model/L2C1_dynamic-26-0.1977.hdf5')
L2C1_combined = load_model('./HAR_LSTM-CNN_extended_model/L2C1_combined-19-0.1407.hdf5')
L2C2_static = load_model('./HAR_LSTM-CNN_extended_model/L2C2_static-33-0.0275.hdf5')
L2C2_dynamic = load_model('./HAR_LSTM-CNN_extended_model/L2C2_dynamic-08-0.2390.hdf5')
L2C2_combined = load_model('./HAR_LSTM-CNN_extended_model/L2C2_combined-24-0.1593.hdf5')

In [59]:
L1C1_static.evaluate(testX_static, testy_static_one_hot)
L1C1_dynamic.evaluate(testX_dynamic, testy_dynamic_one_hot)
L1C1_combined.evaluate(testX_combined, testy_combined_one_hot)
L1C2_static.evaluate(testX_static, testy_static_one_hot)
L1C2_dynamic.evaluate(testX_dynamic, testy_dynamic_one_hot)
L1C2_combined.evaluate(testX_combined, testy_combined_one_hot)
L2C1_static.evaluate(testX_static, testy_static_one_hot)
L2C1_dynamic.evaluate(testX_dynamic, testy_dynamic_one_hot)
L2C1_combined.evaluate(testX_combined, testy_combined_one_hot)
L2C2_static.evaluate(testX_static, testy_static_one_hot)
L2C2_dynamic.evaluate(testX_dynamic, testy_dynamic_one_hot)
L2C2_combined.evaluate(testX_combined, testy_combined_one_hot)

93/93 [==============================] - 3s 27ms/step - loss: 0.5135 - accuracy: 0.8782


[0.5135178565979004, 0.8781812191009521]